In [ ]:
import cv2
import os
import pandas as pd
from datetime import datetime
from serial import Serial
import serial
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
from firebase_admin import credentials, initialize_app, storage
import numpy as np



print(os.getcwd())
#os.chdir('./../')
print(os.getcwd())

In [ ]:
cred = credentials.Certificate("./a-yo-773e1-firebase-adminsdk-dy8v9-728cb5c073.json")
#cred2 = credentials.Certificate("./a-yo-773e1-firebase-adminsdk-dy8v9-728cb5c073.json")
initialize_app(cred, {'databaseURL': 'https://a-yo-773e1-default-rtdb.firebaseio.com', 'storageBucket': 'a-yo-773e1.appspot.com'})
#initialize_app(cred, {'storageBucket': 'a-yo-773e1.appspot.com'})

In [ ]:
arduino = serial.Serial('COM4',9600)

In [ ]:
while True:
    if arduino.in_waiting:
        a = arduino.readline()
        print(a)
        break
    else:
        print("a")

In [ ]:
a = a.decode()

In [ ]:
a

In [ ]:
#아두이노 통신 시
if a == '2\r\n':
    
    webcam = cv2.VideoCapture(0)

    num = 1
    name = []
    lists = []
    lists_jpg = []

    if not webcam.isOpened():
        print("Could not open webcam")
        exit()

    sample_num = 0
    captured_num = 0


    start_time = datetime.now()
    time = str(str(start_time.year) + '_' + str(start_time.month) + '_' + str(start_time.day) +'_' + str(start_time.hour) + '_' + str(start_time.minute) + '_' + str(start_time.second))
    print(time)

    start_sec = start_time.second

    def createFolder(directory):
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSErrorror:
            print ('Error: Creating directory. ' +  directory)
        
    createFolder(time)

    os.chdir(time)
    createFolder('./' + '/original_data')
    createFolder('./' + '/pose_data')


    while webcam.isOpened():
    
        i_time = datetime.now()
        i_sec = i_time.second
    
        status, frame = webcam.read()
        sample_num = sample_num + 1

        if not status:
            print("n")
            break

        cv2.imshow("captured frames", frame)
    

        if sample_num == 100:
            name = "images" + str(num)
            name_jpg = name+".jpg"
            lists.append(name)
            lists_jpg.append(name_jpg)
            cv2.imwrite('original_data/'+lists_jpg[num-1], frame) 
            sample_num = 0
            num += 1
        
        if abs(start_sec - i_sec) >= 28 and abs(start_sec - i_sec) <= 35:
            break

    webcam.release()
    cv2.destroyAllWindows()
    
else:
    print("no")


In [ ]:
print(start_sec)
print(i_sec)

In [ ]:
# fashion_pose.py : MPII를 사용한 신체부위 검출

# MPII에서 각 파트 번호, 선으로 연결될 POSE_PAIRS
BODY_PARTS = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
              "Background": 15}

POSE_PAIRS = [["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
              ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
              ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
              ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"]]

# 각 파일 path
protoFile = "./../src/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "./../src/pose_iter_160000.caffemodel"

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)


for l in range(0,num-1):
    print(num)
    output = []
    
    # 이미지 읽어오기
    print(l)
    print(lists_jpg[l])
    image = cv2.imread('original_data/'+lists_jpg[l])

    # frame.shape = 불러온 이미지에서 height, width, color 받아옴
    imageHeight, imageWidth, _ = image.shape

    # network에 넣기위해 전처리
    inpBlob = cv2.dnn.blobFromImage(image, 1.0 / 255, (imageWidth, imageHeight), (0, 0, 0), swapRB=False, crop=False)

    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()

    # output.shape[0] = 이미지 ID, [1] = 출력 맵의 높이, [2] = 너비
    H = output.shape[2]
    W = output.shape[3]
    print("이미지 ID : ", len(output[0]), ", H : ", output.shape[2], ", W : ", output.shape[3])  # 이미지 ID

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in range(0, 15):
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]

        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (imageWidth * point[0]) / W
        y = (imageHeight * point[1]) / H

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로
        if prob > 0.1:
            print("i = ", i, "x = ", int(x), "y = ", int(y))
            cv2.circle(image, (int(x), int(y)), 3, (0, 255, 255), thickness=-1,
                   lineType=cv2.FILLED)  # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(image, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1,
                    lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
        else:
            print(i)
            points.append(None)

    #cv2.imshow("Output-Keypoints", image)
    #cv2.waitKey(0)

    # 이미지 복사
    imageCopy = image

    # 각 POSE_PAIRS별로 선 그어줌 (머리 - 목, 목 - 왼쪽어깨, ...)
    n = 1
    for pair in POSE_PAIRS:
        partA = pair[0]  # Head
        partA = BODY_PARTS[partA]  # 0
        partB = pair[1]  # Neck
        partB = BODY_PARTS[partB]  # 1

        # print(partA," 와 ", partB, " 연결\n")
        if points[partA] and points[partB]:
            cv2.line(imageCopy, points[partA], points[partB], (0, 255, 0), 2)

    print(points)
    print(type(points))
    print(points[1])

    dict = {}

    a=0
    for key in BODY_PARTS:
        if key == "Background":
            break
        dict[key] = points[a]
        a+=1
        
    print(dict)

    dict = pd.DataFrame(dict)
    #dict.to_excel(excel_writer='1.xlsx')
    
    excel_name =  'excel_data.xlsx' 
    
    if not os.path.exists(excel_name):
        with pd.ExcelWriter(excel_name, mode='w', engine='openpyxl') as writer:
            dict.to_excel(writer, index=False)
    else:
        with pd.ExcelWriter(excel_name, mode='a', engine='openpyxl') as writer:
            dict.to_excel(writer, index=False)

    pose_name = './' +  '/pose_data/' + lists[l] +'_pose.jpg'
    #cv2.imshow("Output-Keypoints", imageCopy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite(pose_name,imageCopy)


In [ ]:
folder_name = time

ref = db.reference('time/')
ref.update({folder_name: folder_name})


In [ ]:
folder_name

In [ ]:
bucket = storage.bucket()
blob = bucket.blob(time + '/pose_data/images1.jpg') #저장할 이름
blob.upload_from_filename('./pose_data/images1_pose.jpg') #저장할 파일
blob.make_public()
print(blob.public_url)

bucket = storage.bucket()
blob = bucket.blob(time + '/pose_data/images2.jpg') #저장할 이름
blob.upload_from_filename('./pose_data/images2_pose.jpg') #저장할 파일
blob.make_public()
print(blob.public_url)

bucket = storage.bucket()
blob = bucket.blob(time + '/pose_data/images3.jpg') #저장할 이름
blob.upload_from_filename('./pose_data/images3_pose.jpg') #저장할 파일
blob.make_public()
print(blob.public_url)

bucket = storage.bucket()
blob = bucket.blob(time + '/pose_data/images4.jpg') #저장할 이름
blob.upload_from_filename('./pose_data/images4_pose.jpg') #저장할 파일
blob.make_public()
print(blob.public_url)

bucket = storage.bucket()
blob = bucket.blob(time + '/pose_data/images5.jpg') #저장할 이름
blob.upload_from_filename('./pose_data/images5_pose.jpg') #저장할 파일
blob.make_public()
print(blob.public_url)

bucket = storage.bucket()
blob = bucket.blob(time + '/pose_data/images6.jpg') #저장할 이름
blob.upload_from_filename('./pose_data/images6_pose.jpg') #저장할 파일
blob.make_public()
print(blob.public_url)

In [ ]:
'''#정석 데이터 기울기 분석

#2-4 오른쪽 어깨 - 오른쪽 손목
inc_2_4 = (dict['RShoulder'][1] - dict['RWrist'][1]) / (dict['RShoulder'][0] - dict['RWrist'][0])
inc_2_4_r = np.round(inc_2_4,5)
print('오른쪽 팔 기울기 = ', inc_2_4)
print('(반올림)오른쪽 팔 기울기 = ', inc_2_4_r)

#5-7 왼쪽 어깨 - 왼쪽 손목
inc_5_7 = (dict['LShoulder'][1] - dict['LWrist'][1]) / (dict['LShoulder'][0] - dict['LWrist'][0])
inc_5_7_r = np.round(inc_5_7, 5)
print('왼쪽 팔 기울기 = ', inc_5_7)
print('(반올림)왼쪽 팔 기울기 = ', inc_5_7_r)

#8-10 오른쪽 엉덩이 오른쪽 무릎
inc_8_10 = (dict['RHip'][1] - dict['RKnee'][1]) / (dict['RHip'][0] - dict['RKnee'][0])
inc_8_10_r = np.round(inc_8_10,5)
print('오른쪽 다리 기울기 = ', inc_8_10)
print('(반올림)오른쪽 다리 기울기 = ', inc_8_10_r)

#11-12 왼쪽 엉덩이 왼쪽 무릎
inc_11_12 = (dict['LHip'][1] - dict['LKnee'][1]) / (dict['LHip'][0] - dict['LKnee'][0])
inc_11_12_r = np.round(inc_11_12,5)
print('왼쪽 다리 기울기 = ', inc_11_12)
print('(반올림)왼쪽 다리 기울기 = ', inc_11_12_r)

inc = {}

inc['RS_RW'] = inc_2_4_r # -0.09105
inc['LS_LW'] = inc_5_7_r # -0.08333
inc['RH_RK'] = inc_8_10_r # -0.97917
inc['Lh_LK'] = inc_11_12_r #0.22222

print(inc)

inc = pd.DataFrame(inc, index=[0])
inc.to_excel(excel_writer='./data/original.xlsx')'''

In [ ]:
#사용자 데이터 기울기 분석
df = pd.read_excel(excel_name)


#2-4 오른쪽 어깨 - 오른쪽 손목
user_2_4 = (df['RShoulder'][1] - df['RWrist'][1]) / (df['RShoulder'][0] - df['RWrist'][0])
user_2_4_r = np.round(user_2_4,5)
print('오른쪽 팔 기울기 = ', user_2_4)
print('(반올림)오른쪽 팔 기울기 = ', user_2_4_r)

#5-7 왼쪽 어깨 - 왼쪽 손목
user_5_7 = (df['LShoulder'][1] - df['LWrist'][1]) / (df['LShoulder'][0] - df['LWrist'][0])
user_5_7_r = np.round(user_5_7, 5)
print('왼쪽 팔 기울기 = ', user_5_7)
print('(반올림)왼쪽 팔 기울기 = ', user_5_7_r)


#11-12 왼쪽 엉덩이 왼쪽 무릎
user_11_12 = (df['LHip'][1] - df['LKnee'][1]) / (df['LHip'][0] - df['LKnee'][0])
user_11_12_r = np.round(user_11_12,5)
print('왼쪽 다리 기울기 = ', user_11_12)
print('(반올림)왼쪽 다리 기울기 = ', user_11_12_r)

user = {}

user['RS_RW'] = user_2_4_r 
user['LS_LW'] = user_5_7_r 
user['Lh_LK'] = user_11_12_r 

print(user)

#inc = pd.DataFrame(inc, index=[0])
#inc.to_excel(excel_writer='./data/original.xlsx') 

In [ ]:
inc_2_4_r = -0.09105
inc_5_7_r = -0.08333
inc_11_12_r = 0.22222

user_2_4 = -0.09000
user_5_7 = -0.05000
user_11_12 = 0.20000

In [ ]:
if inc_2_4_r > user_2_4 : #사용자 기울기가 작다면(절대값이 크다면) 오른쪽 팔을 더 내려주세요
    c = 'a'
    c = c.encode('utf-8')
    arduino.write(c)

if inc_2_4_r < user_2_4 : #사용자 기울기가 크다면(절대값이 작다면) 오른쪽 팔을 더 올려주세요
    c = 'b'
    c = c.encode('utf-8')
    arduino.write(c)

#if inc_2_4 == user_2_4 : #사용자 기울기가 같다면 아무말 안함

if inc_5_7_r > user_5_7 : #사용자 기울기가 작다면(절대값이 크다면) 왼쪽 팔을 더 내려주세요
    c = 'c'
    c = c.encode('utf-8')
    arduino.write(c)

if inc_5_7_r > user_5_7 : #사용자 기울기가 크다면(절대값이 작다면) 왼쪽 팔을 더 올려주세요
    c = 'd'
    c = c.encode('utf-8')
    arduino.write(c)

#if inc_5_7 == user_5_7 : #사용자 기울기가 같다면 아무말 안함
    
    
if inc_11_12_r < user_11_12 : #사용자 기울기가 크다면 왼쪽 무릎을 직각에 가깝게 앉아주세요
    c = 'e'
    c = c.encode('utf-8')
    arduino.write(c)
    
#if inc_11_12 == user_11_12 : #사용자 기울기가 같다면 아무말 안함